In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import random
import re
import time
from rich import print
import requests
chrome_driver_path="/usr/local/bin/chromedriver"

In [2]:
# Aumentamos timeout de verificación de proxy
def test_proxy(proxy_ip):
    try:
        # Verificamos contra un sitio real (no solo httpbin)
        test_urls = [
            "https://www.google.com",
            "https://www.autocompara.com"
        ]
        for url in test_urls:
            response = requests.get(
                url,
                proxies={"http": proxy_ip, "https": proxy_ip},
                timeout=15  # Aumentamos timeout
            )
            if not response.ok:
                return False
        return True
    except Exception:
        return False

def init_web(ip: str):
    options = Options()
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1366,768")
    
    # Validación mejorada del proxy
    if ip:
        if test_proxy(ip):
            options.add_argument(f"--proxy-server={ip}")
            print(f"[green]Proxy activo: {ip}[/]")
        else:
            print(f"[red]Proxy falló en pruebas: {ip}[/]")
            ip = None
    if not ip:
        print("[yellow]Usando conexión directa[/]")

    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    
    # Ajustamos timeouts del navegador
    driver.set_page_load_timeout(60)  # 60 segundos para carga
    driver.set_script_timeout(30)     # 30 segundos para scripts
    
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        window.chrome = {runtime: {}};
        """
    })
    
    try:
        # Intentamos conectar con timeout extendido
        driver.get("https://www.autocompara.com")
        
        # Espera adaptativa para desafíos de seguridad
        WebDriverWait(driver, 30).until_not(
            lambda d: any(
                kw in d.title.lower() 
                for kw in ["cloudflare", "security", "captcha", "access denied"]
            )
        )
    except TimeoutException:
        print("[yellow]Advertencia: Tiempo de espera extendido alcanzado[/]")
    except Exception as e:
        print(f"[red]Error en navegación: {str(e)}[/]")
        driver.quit()
        return None
    
    try:
        # Intento más robusto para eliminar overlays
        overlay_selectors = [
            (By.ID, "sec-overlay"),
            (By.CSS_SELECTOR, "div.overlay, div.modal, div.security-layer")
        ]
        for selector in overlay_selectors:
            try:
                WebDriverWait(driver, 5).until(EC.visibility_of_element_located(selector))
                driver.execute_script(f"document.querySelector('{selector[1]}').remove();")
            except:
                continue
    except Exception:
        pass
    
    return driver

In [3]:

def insert_year(driver, año: int):
  try:
    input_year=WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "year")))
    time.sleep(0.5)
    input_year.clear()
    for char in str(año):
      input_year.send_keys(char)
      time.sleep(0.1)
    print(f"[bold cyan]año {año} insertado correctamente[/]")
    return True
  except TimeoutException:
    print("[bold red]tiempo de espera agotado, no se encontro el elemento con id 'year'[/]")
    return False
  except NoSuchElementException:
    print("[bold red]el elemento con id 'year' no existe en la pagina[/]")
    return False
  except Exception as e:
    print(f"[bold red]error inesperado: {str(e)}[/]")
    return False

In [4]:
def insert_model(driver, texto: str):
  try:
    input_container=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ng-input")))

    driver.execute_script(
      """arguments[0].scrollIntoView({
        behavior: 'auto',
        block: 'center',
        inline: 'center'
      });""",
      input_container
    )
    time.sleep(0.5)

    driver.execute_script("arguments[0].click();", input_container)

    input_field=WebDriverWait(driver, 5).until(
      EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ng-input input"))
    )

    value=texto.upper() 
    for char in texto:
      input_field.send_keys(char)
      time.sleep(random.uniform(0.1, 0.25))
      current_value = input_field.get_attribute("value").upper()
      if char not in current_value:
        raise ValueError(f"Carácter '{char}' no detectado. Valor actual: {current_value}")
      
    WebDriverWait(driver, 10).until(lambda d: texto in d.find_element(By.CSS_SELECTOR, "div.ng-input input").get_attribute("value").upper())

    print(f"[bold cyan]busqueda exitosa para: {value}[/]")
    return True
  
  except Exception as e:
    print(f"[bold red]fallo en busqueda: {str(e)}")
    try:
      driver.execute_script("""
        document.querySelector('ng-select#search').click();
        document.querySelector('ng-select#search input').value = arguments[0];
        document.querySelector('ng-select#search input').dispatchEvent(new Event('input'));
      """, texto)
      print(f"[bold yellow]usando fallback JS para: {texto}[/]")
      return True
    except:
      return False

In [5]:
def select_model(driver, texto: str):
  try:
    WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ng-dropdown-panel[id^='a']")))

    options = driver.find_elements(By.CSS_SELECTOR, "div.ng-option-child:not(.ng-optgroup)")

    find=False
    for option in options:
      try:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", option)
        texto_option=option.find_element(By.CSS_SELECTOR, "div.vehicle-data-home").get_attribute("textContent").strip()
        if texto_option.lower()==texto.lower():
          driver.execute_script("arguments[0].click();", option)
          print(f"[bold cyan]opcion seleccionada: {texto_option}[/]")
          find=True
          break
      except Exception as e:
        print(f"[bold red]error durante iteracion de opciones: {str(e)}[/]")
        continue
    if not find:
      print("[bold yellow]Opción no encontrada, usando alternativa...[/]")
      options[0].click()
    return True
  except Exception as e:
    print(f"[bold red]error general: {str(e)}[/]")
    return False

In [6]:
def seleccionar_opcion_alternativa(driver, texto: str):
  try:
    option=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
      By.XPATH,
      f"//div[contains(@class, 'ng-option') and not(contains(@class, 'ng-optgroup'))]//div[contains(@class, 'vehicle-data-home') and normalize-space()='{texto}']/ancestor::div[contains(@class, 'ng-option')]"
    )))
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center', inline: 'nearest'});", option)
    time.sleep(0.3)
    driver.execute_script("arguments[0].click();", option)
    return True
  except Exception as e:
    print(f"[bold red]error en metodo alternativo: {str(e)}[/]")
    return False

In [7]:
def click_continuar(driver):
  try:
    boton=WebDriverWait(driver, 15).until(EC.element_to_be_clickable((
      By.XPATH, 
      "//button[contains(@class, 'btn-gradient') and .//text()[contains(., 'Continuar')]]"
    )))
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton)
    driver.execute_script("arguments[0].click();", boton)
    print("[bold cyan]boton continuar clickeado exitosamente[/]")
    return True
  except Exception as e:
    print(f"[bold red]error al hacer clic en continuar: {str(e)}[/]")
    return False

In [8]:
def select_gender(driver, genero: str):
  try:
    selectores={
      "Hombre": {
        "xpath": "//label[contains(@class, 'male') and normalize-space()='Hombre']",
        "css": "label.male"
      },
      "Mujer": {
        "xpath": "//label[contains(@class, 'female') and normalize-space()='Mujer']",
        "css": "label.female"
      }
    }
    if genero not in selectores:
      raise ValueError(f"[bold purple]opcion invalida: {genero}. Use 'Hombre' o 'Mujer'[/]")
    elemento=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
      By.XPATH, 
      selectores[genero]["xpath"]
    )))
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", elemento)
    driver.execute_script("arguments[0].click();", elemento)
    input_radio=elemento.find_element(By.TAG_NAME, "input")
    if input_radio.is_selected():
      print(f"[bold cyan]genero seleccionado correctamente {genero}[/]")
      return True
    else:
      print(f"[bold red]error, el radio button no se marco[/]")
      return False
  except Exception as e:
    print(f"[bold red]error seleccionando genero: {str(e)}[/]")
    try:
      elemento=driver.find_element(By.CSS_SELECTOR, selectores[genero]["css"])
      elemento.click()
      return True
    except:
      print("[bold red]error con el selector en css[/]")
      return False

In [9]:
def insert_date(driver, fecha: str):
  try:
    campo_fecha=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "date")))
    driver.execute_script("arguments[0].value = '';", campo_fecha)
    campo_fecha.clear()
    time.sleep(0.5)
    for i, char in enumerate(fecha):
      campo_fecha.send_keys(char)
      current_value=campo_fecha.get_attribute("value")
      if i in [2, 5]:
        if len(current_value) <= i or current_value[i] != "/":
          driver.save_screenshot(f"error_formato_{i}.png")
          raise ValueError(f"[bold red]Formato inválido en posición {i+1}")
      time.sleep(0.1)
    WebDriverWait(driver, 5).until(lambda d: d.find_element(By.ID, "date").get_attribute("value").replace("/", "") == fecha.replace("/", ""))
    print(f"[bold cyan]fecha ingresada correctamente {fecha}[/]")
    return True
  except ElementNotInteractableException:
    driver.execute_script(
      """
      const input = document.getElementById('date');
      input.value = arguments[0];
      input.dispatchEvent(new Event('input', { bubbles: true }));
      input.dispatchEvent(new Event('change', { bubbles: true }));
      """,
      fecha 
    )
    print(f"[bold purple]fecha ingresada mediante JavaScript[/]")
    return True
  except Exception as e:
    print(f"[bold red]error ingresando fecha: {str(e)}")
    driver.save_screenshot("error_fecha.png")
    return False

In [10]:
def insert_personal_data(driver, nombre: str, codigo_p: str, email: str, telefono: str):
  try:
    campos={
      "name": {
        "valor": nombre,
        "validacion": lambda x: len(x.split()) >= 1
      },
      "cp": {
        "valor": codigo_p,
        "validacion": lambda x: x.isdigit() and len(x) == 5
      },
      "email": {
        "valor": email,
        "validacion": lambda x: re.match(r"[^@]+@[^@]+\.[^@]+", x)
      },
      "phone": {
        "valor": telefono,
        "validacion": lambda x: x.isdigit() and len(x) in [10, 12]
      }
    }
    for campo_id, config in campos.items():
      if not config["validacion"](config["valor"]):
        raise ValueError(f"[bold red]formato invalido para {campo_id}: {config['valor']}")
      elemento=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, campo_id)))
      elemento.clear()
      for char in config["valor"]:
        elemento.send_keys(char)
        time.sleep(0.05)
    print("[bold cyan]todos los campos llenados correctamente")
    return True
  except Exception as e:
    print(f"[bold red]error en llenado de datos: {str(e)}")
    driver.save_screenshot("error_datos_personales.png")
    return False

In [11]:
def click_cotizar(driver):
  try:
    boton=WebDriverWait(driver, 25).until(EC.element_to_be_clickable((
      By.XPATH, 
      "//button[contains(@class, 'btn-gradient') and contains(., 'Cotizar')]"
    )))
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton)
    time.sleep(random.uniform(0.5, 1.5))
    driver.execute_script("arguments[0].click();", boton)
    print("[bold cyan]click en cotizar realizado[/]")
    try:
      WebDriverWait(driver, 40).until(lambda d: any([
        len(d.find_element(By.CSS_SELECTOR, "div.resultado-cotizacion"))>0,
        "resultado" in d.current_url,
        d.execute_script("return document.readyState")=="complete"
      ]))
    except:
      time.sleep(10)
      if "cotizacion" not in driver.current_url:
        raise TimeoutError("[bold red] la pagina no termino de cargar[/]")
    
    print("[bold cyan]pagina de resultados cargada exitosamente")
    return True
  except Exception as e:
    print(f"[bold red]error final: {str(e)}[/]")
    print("[bold purple]estado actual:[/]")
    print(f"[bold purple]- URL: {driver.current_url}[/]")
    print(f"[bold purple]- titulo: {driver.title}[/]")
    print(f"[bold purple]- ReadyState: {driver.execute_script('return document.readyState')}[/]")
    driver.save_screenshot("error_final.png")
    return False

In [12]:
def abrir_modal_coberturas(driver):
  try:
    WebDriverWait(driver, 15).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    boton_selector = (
        "button.btn.btn-lnk.details[acgtmevent][category='compara_y_elige'][action='informacion_seguros']"
    )
    boton = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, boton_selector)))
    driver.execute_script(
        "arguments[0].scrollIntoView({behavior: 'auto', block: 'center', inline: 'center'});"
        "window.scrollBy(0, -window.innerHeight * 0.1);", 
        boton
    )
    time.sleep(random.uniform(0.3, 0.7))
    try:
        ActionChains(driver)\
            .move_to_element_with_offset(boton, 5, 5)\
            .pause(random.uniform(0.1, 0.3))\
            .click()\
            .perform()
    except:
        driver.execute_script("arguments[0].click();", boton)
    WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.modal-content"))
    )
    WebDriverWait(driver, 10).until(
        lambda d: "coberturas" in d.page_source.lower()
    )
    print("[bold green]Modal de coberturas abierto correctamente[/]")
    return True
  except TimeoutException:
    print("[bold red]Tiempo de espera excedido para el modal[/]")
    driver.save_screenshot("error_modal_timeout.png")
    return False
      
  except Exception as e:
    print(f"[bold red]Error abriendo modal: {str(e)}[/]")
    driver.save_screenshot("error_modal_general.png")
    try:
        driver.execute_script("""
            const modals = document.querySelectorAll('div.modal-backdrop');
            modals.forEach(modal => modal.remove());
            document.body.classList.remove('modal-open');
        """)
    except:
        pass
    
    return False

In [13]:
def interactuar_ng_select(driver, intentos=3):
    try:
        selector_ng_select = "ng-select[name='insuaranse'].custom"
        ng_select = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, selector_ng_select))
        )
        trigger_selector = "div.ng-input[role='combobox']"
        trigger = ng_select.find_element(By.CSS_SELECTOR, trigger_selector)
        driver.execute_script("""
            const element = arguments[0];
            const header = document.querySelector('header') || { offsetHeight: 0 };
            const yOffset = -header.offsetHeight * 0.15;
            
            element.scrollIntoView({
                behavior: 'auto',
                block: 'center',
                inline: 'nearest'
            });
            
            window.scrollBy(0, yOffset);
        """, trigger)
        for intento in range(intentos):
            try:
                ActionChains(driver)\
                    .move_to_element_with_offset(trigger, 5, 5)\
                    .pause(0.25)\
                    .click()\
                    .pause(0.5)\
                    .perform()
                if trigger.get_attribute("aria-expanded") == "true":
                    print("[green]Dropdown desplegado[/]")
                    return True
                if "ng-select-opened" in ng_select.get_attribute("class"):
                    print("[green]Dropdown abierto (verificación por clase)[/]")
                    return True
                print(f"[yellow]Intento {intento + 1} - Dropdown no se abrió[/]")
            except StaleElementReferenceException:
                print("[yellow]Elemento obsoleto, reintentando...[/]")
                trigger = ng_select.find_element(By.CSS_SELECTOR, trigger_selector)
                continue
        panel_selector = "ng-dropdown-panel.custom"
        if EC.visibility_of_element_located((By.CSS_SELECTOR, panel_selector))(driver):
            print("[green]panel detectado post-intentos[/]")
            return True
        raise Exception("No se pudo desplegar el dropdown después de 3 intentos")
    except Exception as e:
        print(f"[red]Error crítico: {str(e)}[/]")
        driver.save_screenshot("error_ng_select.png")
        return False

In [14]:
def obtener_opciones_completas(driver):
    try:
        panel_selector = "ng-dropdown-panel.custom"
        panel = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, panel_selector)))
        opciones = panel.find_elements(By.CSS_SELECTOR, "div.ng-option[id]")
        opciones_data = []
        for opcion in opciones:
            try:
                opcion_id = opcion.get_attribute("id")
                nombre = opcion.find_element(By.CSS_SELECTOR, "span.ng-option-label").text.strip()
                if opcion_id and "-" in opcion_id and nombre:
                    opciones_data.append((opcion_id, nombre))
                    
            except Exception as e:
                print(f"[yellow]Error en opción: {str(e)}[/]")
                continue
        
        print(f"[green]{len(opciones_data)} opciones encontradas[/]")
        
        return opciones_data
    
    except Exception as e:
        print(f"[red]Error general: {str(e)}[/]")
        driver.save_screenshot("error_opciones_completas.png")
        return []

In [15]:
def seleccionar_opcion_por_id(driver, option_id: str):
    try:
        print(f"[bold cyan]Procesando opción ID: {option_id}[/]")
        partial_id = option_id.split("-")[-1]
        panel_selector = "ng-dropdown-panel.custom"
        for _ in range(2):
            try:
                WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, panel_selector)))
                break
            except:
                print("[yellow]Reabriendo dropdown...[/]")
                if not interactuar_ng_select(driver):
                    raise Exception("No se pudo reabrir el dropdown")
        xpath_selector = f"//div[@role='option' and substring(@id, string-length(@id) - {len(partial_id)} + 1) = '{partial_id}']"
        opcion = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, xpath_selector))
        )
        ActionChains(driver)\
            .move_to_element(opcion)\
            .perform()
        
        time.sleep(0.2)
        for intento in range(3):
            try:
                driver.execute_script("arguments[0].scrollIntoViewIfNeeded()", opcion)
                opcion.click()
                WebDriverWait(driver, 2).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, panel_selector))
                )
                print(f"[green]Selección confirmada: {option_id}[/]")
                return True
            except StaleElementReferenceException:
                print(f"[yellow]Reintento {intento+1} - Actualizando referencia...[/]")
                opcion = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, xpath_selector))
                )
            except Exception as e:
                print(f"[yellow]Error en intento {intento+1}: {str(e)}[/]")
                driver.execute_script("arguments[0].click()", opcion)
        raise Exception("Máximo de reintentos alcanzado")
    except Exception as e:
        print(f"[red]Fallo en {option_id}: {str(e)}[/]")
        driver.save_screenshot(f"error_{option_id.replace('-', '_')}.png")
        return False

In [16]:
import re
def obtener_precio_anual(driver) -> str:
    try:
        xpath_selector = (
            ".//p[contains(@class, 'value') "
            "and not(*[not(self::text())]) "
            "and contains(., '$')]"
        )
        precios = WebDriverWait(driver, 15).until(
            lambda d: [
                p for p in d.find_elements(By.XPATH, xpath_selector)
                if p.is_displayed() 
                and not {'valueB'}.intersection(p.get_attribute("class").split())
                and re.search(r"\$\s*\d", p.text)
            ]
        )
        elemento_correcto = None
        for p in precios:
            classes = p.get_attribute("class").split()
            if 'valueA' in classes:
                elemento_correcto = p
                break
            if 'value' in classes and not {'valueA', 'valueB'}.intersection(classes):
                elemento_correcto = p
                break
        if not elemento_correcto:
            elemento_correcto = precios[0] if precios else None
            print("[yellow]Usando fallback de precio[/]")
        if not elemento_correcto or elemento_correcto.find_elements(By.XPATH, "./*"):
            raise ValueError("Estructura de precio inválida")
        precio_texto = elemento_correcto.text
        match = re.search(
            r"""
            \$
            \s*
            (
                \d{1,3}
                (?:,\d{3})*
                (?:\.\d{2})?
            )
            """, 
            precio_texto, 
            re.VERBOSE
        )
        if not match:
            raise ValueError(f"Formato numérico inválido: {precio_texto}")
        precio_limpio = match.group(1).replace(",", "")
        precio_final = float(precio_limpio)
        
        return f"{precio_final:.2f}"
    except Exception as e:
        print(f"[red]Error crítico: {str(e)}[/]")
        driver.save_screenshot("error_precio_estricto.png")
        return None

In [17]:
def expandir_todos_elementos(driver, timeout=20, pause_between=0.5, retry_attempts=3):
    xpath_icono = (
        "//div[contains(@class, 'row-down-propertie')]"
        "//em[contains(@class, 'icon-angle-down-af')]"
        "/ancestor::div[contains(@class, 'row-down-propertie')]"
    )
    try:
        elementos = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath_icono))
        )
    except TimeoutException:
        print("[yellow]No se encontraron elementos para expandir.[/]")
        return False
    total = len(elementos)
    print(f"[cyan]Encontrados {total} elementos colapsados.[/]")
    for index in range(total):
        for attempt in range(1, retry_attempts + 1):
            try:
                elementos = driver.find_elements(By.XPATH, xpath_icono)
                if index >= len(elementos):
                    break
                boton = elementos[index]
                driver.execute_script(
                    "const header = document.querySelector('header') || { offsetHeight: 0 };"
                    "arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});"
                    "window.scrollBy(0, -header.offsetHeight);",
                    boton
                )
                WebDriverWait(driver, timeout).until(
                    EC.element_to_be_clickable((By.XPATH, xpath_icono))
                )
                ActionChains(driver).move_to_element(boton).pause(pause_between).click().perform()
                time.sleep(pause_between)
                break
            except (StaleElementReferenceException, ElementClickInterceptedException, TimeoutException) as e:
                if attempt < retry_attempts:
                    print(f"[yellow]Reintentando elemento {index+1}, intento {attempt}/{retry_attempts}...[/]")
                    time.sleep(pause_between)
                    continue
                else:
                    print(f"[red]No se pudo expandir el elemento {index+1} tras {retry_attempts} intentos: {e}[/]")
    print(f"[bold green]Proceso completado: {total} intentos realizados.[/]")
    return True

In [18]:
def extraer_danios_materiales_2(driver):
    try:
        contenedor_principal = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, "card-1"))
        )
        seccion_coberturas = WebDriverWait(contenedor_principal, 10).until(
            lambda d: d.find_element(
                By.XPATH,
                ".//p[@class='subtitle mb-0' and contains(text(), 'Coberturas')]/ancestor::div[contains(@class, 'cont-servicios')]"
            )
        )
        coberturas = seccion_coberturas.find_elements(
            By.XPATH,
            ".//div[contains(@class, 'centrado')]/p[@class='description']"
        )
        lista_coberturas = []
        for elemento in coberturas:
            texto = elemento.text.strip()
            if texto and texto not in lista_coberturas:
                lista_coberturas.append(texto)
        return lista_coberturas
    
    except Exception as e:
        print(f"[red]Error extrayendo coberturas: {str(e)}[/]")
        driver.save_screenshot("error_coberturas.png")
        return []

In [19]:
import uuid

def extract_informacion(driver, aseguradoras):
  id_data=str(uuid.uuid4())
  total_data={}
  for i, elemento in enumerate(aseguradoras):
    if i!=0:
      interactuar_ng_select(driver)
      seleccionar_opcion_por_id(driver, elemento[0])
    precio=obtener_precio_anual(driver)
    expandir_todos_elementos(driver)
    data=extraer_danios_materiales_2(driver)
    total_data[elemento[1]]={
      "precio": precio,
      "daños_materiales": data,
    }

  return {f"{id_data}": total_data}

In [20]:
ip="132.148.21.113:27593"
driver=init_web(ip)

Proxy falló en pruebas: 132.148.21.113:27593

Usando conexión directa

In [21]:
insert_year(driver, 2015)
insert_model(driver, "AVEO LS A STD 1.6L 4CIL 4PTAS")
select_model(driver, "AVEO LS A STD 1.6L 4CIL 4PTAS")
click_continuar(driver)

año 2015 insertado correctamente

fallo en busqueda: Message: 

usando fallback JS para: AVEO LS A STD 1.6L 4CIL 4PTAS

opcion seleccionada: Aveo Ls A Std 1.6l 4cil 4ptas

boton continuar clickeado exitosamente

True

In [22]:
select_gender(driver,"Hombre")
insert_date(driver, "16072002")
insert_personal_data(driver, "Emilio", "52977", "foyagev912@ofular.com", "524385654784")
click_cotizar(driver)

genero seleccionado correctamente Hombre

fecha ingresada correctamente 16072002

todos los campos llenados correctamente

click en cotizar realizado

pagina de resultados cargada exitosamente

True

In [23]:
abrir_modal_coberturas(driver)
interactuar_ng_select(driver)
aseguradoras=obtener_opciones_completas(driver)
print(aseguradoras)

Modal de coberturas abierto correctamente

Dropdown desplegado

11 opciones encontradas

[
    ('af064f355456-0', 'CHUBB'),
    ('af064f355456-1', 'HDI'),
    ('af064f355456-2', 'QUÁLITAS'),
    ('af064f355456-3', 'INBURSA'),
    ('af064f355456-4', 'ZURICH'),
    ('af064f355456-5', 'ANA'),
    ('af064f355456-6', 'ATLAS'),
    ('af064f355456-7', 'GNP'),
    ('af064f355456-8', 'MAPFRE'),
    ('af064f355456-9', 'AIG'),
    ('af064f355456-10', 'AXA')
]

In [24]:
seleccionar_opcion_por_id(driver, "a9258ab73c6c-1")
print(obtener_precio_anual(driver))

Procesando opción ID: a9258ab73c6c-1

Selección confirmada: a9258ab73c6c-1

10952.14

In [25]:
expandir_todos_elementos(driver)
data=extraer_danios_materiales_2(driver)
print(data)

Encontrados 8 elementos colapsados.

Proceso completado: 8 intentos realizados.

[
    'Colisiones, vuelcos y estrellamientos',
    'Rotura de cristales',
    'Incendio, rayo y explosión',
    'Catástrofes naturales',
    'Paros, mítines, huelgas o disturbios',
    'Daños al transportar el vehículo asegurado',
    'Penetración de agua al motor'
]

In [26]:
print(extract_informacion(driver, aseguradoras))

Encontrados 8 elementos colapsados.

Proceso completado: 8 intentos realizados.

Dropdown desplegado

Procesando opción ID: af064f355456-1

Error en intento 1: Message: 

...

Procesando opción ID: af064f355456-10

Selección confirmada: af064f355456-10

Encontrados 8 elementos colapsados.

Proceso completado: 8 intentos realizados.

{
    'c6583343-af80-4efb-a3b3-43af08b697d1': {
        'CHUBB': {'precio': '10952.14', 'daños_materiales': []},
        'HDI': {
            'precio': '10952.14',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'QUÁLITAS': {
            'precio': '9927.00',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'INBURSA': {
            'precio': '11422.60',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'ZURICH': {
            'precio': '11772.96',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'ANA': {
            'precio': '12057.34',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'ATLAS': {
            'precio': '12104.15',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'GNP': {
            'precio': '12133.23',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'MAPFRE': {
            'precio': '14943.02',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio, rayo y explosión',
                'Catástrofes naturales',
                'Paros, mítines, huelgas o disturbios',
                'Daños al transportar el vehículo asegurado',
                'Penetración de agua al motor'
            ]
        },
        'AIG': {
            'precio': '17114.73',
            'daños_materiales': [
                'Colisiones, vuelcos y estrellamientos',
                'Rotura de cristales',
                'Incendio,

In [ ]:
def extraer_tiempo_pago_robo(driver):
    try:
        # Localizar la sección por el título específico
        seccion_tiempo_pago = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//p[@class='subtitle mb-0' and contains(text(), 'Tiempo de pago de indemnización')]"
                "/ancestor::div[contains(@class, 'cont-servicios')]"
            ))
        )
        
        # Extraer el texto del tiempo de pago
        tiempo_pago = seccion_tiempo_pago.find_element(
            By.XPATH,
            ".//p[@class='description']"
        ).text.strip()
        
        return tiempo_pago
    
    except Exception as e:
        
        return None

In [31]:
timepo_pago=extraer_tiempo_pago_robo(driver)
print(timepo_pago)

5 días

In [32]:
def extraer_extension_responsabilidad_civil(driver):
    try:
        # Esperar a que el título sea visible
        WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((
                By.XPATH, 
                "//p[contains(@class, 'subtitle') and contains(text(), 'Extensión de Responsabilidad Civil')]"
            ))
        )
        
        # Localizar la columna de contenido específica
        columna_contenido = driver.find_element(
            By.XPATH,
            "//p[@class='subtitle mb-0' and contains(text(), 'Extensión de Responsabilidad Civil')]"
            "/ancestor::div[contains(@class, 'cont-servicios')]"
            "//div[contains(@class, 'col-5')]"
        )
        
        # Extraer todos los elementos de descripción
        elementos = columna_contenido.find_elements(By.XPATH, ".//p[@class='description']")
        
        return list({elem.text.strip() for elem in elementos if elem.text.strip()})
        
    except TimeoutException:
        print("[yellow]Sección de Responsabilidad Civil no encontrada - puede que no esté presente[/]")
        return []
    except NoSuchElementException:
        print("[yellow]Elementos específicos no encontrados en la sección[/]")
        return []
    except Exception as e:
        print(f"[red]Error inesperado: {str(e)}[/]")
        driver.save_screenshot("error_inesperado_responsabilidad.png")
        return []

In [33]:
responsabilidad_civil=extraer_extension_responsabilidad_civil(driver)
print(responsabilidad_civil)

[
    'Asistencia Legal',
    'Daños materiales a terceros',
    'Asistencia Vial',
    'Te protege aún cuando manejes otro vehículo que no este asegurado y sea de similares características al 
amparado'
]

In [40]:
def extraer_gastos_medicos(driver):
    try:
        # Localizar la sección específica con el título "Coberturas" centrado
        seccion_gastos_medicos = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//div[contains(@class, 'cont-servicios') and .//div[contains(@class, 'd-flex justify-content-center') and .//p[text()='Coberturas']]]"
            ))
        )
        
        # Extraer todas las coberturas médicas de la columna específica
        coberturas = seccion_gastos_medicos.find_elements(
            By.XPATH,
            ".//div[contains(@class, 'col-5')]//p[@class='description']"
        )
        
        # Procesar los textos
        lista_coberturas = [elemento.text.strip() for elemento in coberturas if elemento.text.strip()]
        return list(set(lista_coberturas))  # Eliminar duplicados si los hay
    
    except Exception as e:
        print(f"[red]Error extrayendo gastos médicos: {str(e)}[/]")
        driver.save_screenshot("error_gastos_medicos.png")
        return []

In [41]:
medicos=extraer_gastos_medicos(driver)
print(medicos)

['Hospitalización', 'Servicio de ambulancia', 'Atención médica', 'Gastos de entierro', 'Medicinas', 'Enfermeros']

In [46]:
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

def json_to_excel(json_data, output_file):
    # Obtener el ID principal y los datos
    cotizacion_id = list(json_data.keys())[0]
    datos_generales = json_data[cotizacion_id]
    
    # Crear un nuevo libro de Excel
    wb = Workbook()
    
    # ======================================================
    # Hoja 1: Detalles Generales
    # ======================================================
    if 'ws' in wb.sheetnames:
        ws_general = wb['ws']
        wb.remove(ws_general)
    ws_general = wb.create_sheet("Detalles Generales")
    
    # Preparar datos generales
    general_data = {
        "ID": [cotizacion_id],
        "Versión Autocompara": [datos_generales["Versión Autocompara "]],
        "Carrocerías": [datos_generales["Carrocerías"]],
        "Tipo": [datos_generales["Tipo"]],
        "Descripción": [datos_generales["Desc"]],
        "Año Modelo": [datos_generales["Año Mod"]],
        "CP": [datos_generales["CP"]],
        "Ciudad": [datos_generales["Ciudad"]],
        "Entidad": [datos_generales["Entidad"]],
        "Género": [datos_generales["Género"]],
        "Edad": [datos_generales["Edad"]],
        "Fecha Nacimiento": [datos_generales["Fecha Nacimiento"]],
    }
    
    # Añadir precios de aseguradoras
    aseguradoras = ["CHUBB", "HDI", "ATLAS", "ZURICH", "MAPFRE", 
                   "GNP", "INBURSA", "ANA", "AIG", "QUÁLITAS", "AXA"]
    
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            precio = datos_generales[aseguradora]["precio"]
            general_data[aseguradora] = [float(precio)]
        else:
            general_data[aseguradora] = [None]
    
    # Crear DataFrame y escribir en hoja
    df_general = pd.DataFrame(general_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_general, index=False, header=True), 1):
        ws_general.append(row)
    
    # Formatear encabezados
    for cell in ws_general[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Hoja 2: Resumen Coberturas
    # ======================================================
    ws_resumen = wb.create_sheet("Resumen Coberturas")
    
    # Preparar datos para resumen
    resumen_data = []
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            datos_aseguradora = datos_generales[aseguradora]
            resumen_data.append({
                "ID": cotizacion_id,
                "Aseguradora": aseguradora,
                "daños_materiales": len(datos_aseguradora["daños_materiales"]),
                "responsabilidad_civil": len(datos_aseguradora["responsabilidad_civil"]),
                "gastos_medicos": len(datos_aseguradora["gastos_medicos"]),
                "tiempo_robo": datos_aseguradora["tiempo_robo"] if datos_aseguradora["tiempo_robo"] else None
            })
    
    # Crear DataFrame y escribir en hoja
    df_resumen = pd.DataFrame(resumen_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_resumen, index=False, header=True), 1):
        ws_resumen.append(row)
    
    # Formatear encabezados
    for cell in ws_resumen[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Hoja 3: Daños Materiales
    # ======================================================
    ws_danios = wb.create_sheet("Daños Materiales")
    
    # Preparar datos
    danios_data = []
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            danios = datos_generales[aseguradora]["daños_materiales"]
            for cobertura in danios:
                danios_data.append({
                    "ID": cotizacion_id,
                    "Aseguradora": aseguradora,
                    "Cobertura": cobertura
                })
    
    # Crear DataFrame y escribir en hoja
    df_danios = pd.DataFrame(danios_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_danios, index=False, header=True), 1):
        ws_danios.append(row)
    
    # Formatear encabezados
    for cell in ws_danios[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Hoja 4: Tiempo de Robo
    # ======================================================
    ws_robo = wb.create_sheet("Tiempo Robo")
    
    # Preparar datos
    robo_data = []
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            tiempo = datos_generales[aseguradora]["tiempo_robo"]
            robo_data.append({
                "ID": cotizacion_id,
                "Aseguradora": aseguradora,
                "Tiempo Robo": tiempo if tiempo else "No especificado"
            })
    
    # Crear DataFrame y escribir en hoja
    df_robo = pd.DataFrame(robo_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_robo, index=False, header=True), 1):
        ws_robo.append(row)
    
    # Formatear encabezados
    for cell in ws_robo[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Hoja 5: Responsabilidad Civil
    # ======================================================
    ws_responsabilidad = wb.create_sheet("Responsabilidad Civil")
    
    # Preparar datos
    responsabilidad_data = []
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            coberturas = datos_generales[aseguradora]["responsabilidad_civil"]
            for cobertura in coberturas:
                responsabilidad_data.append({
                    "ID": cotizacion_id,
                    "Aseguradora": aseguradora,
                    "Cobertura": cobertura
                })
    
    # Crear DataFrame y escribir en hoja
    df_responsabilidad = pd.DataFrame(responsabilidad_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_responsabilidad, index=False, header=True), 1):
        ws_responsabilidad.append(row)
    
    # Formatear encabezados
    for cell in ws_responsabilidad[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Hoja 6: Gastos Médicos
    # ======================================================
    ws_gastos = wb.create_sheet("Gastos Médicos")
    
    # Preparar datos
    gastos_data = []
    for aseguradora in aseguradoras:
        if aseguradora in datos_generales:
            coberturas = datos_generales[aseguradora]["gastos_medicos"]
            for cobertura in coberturas:
                gastos_data.append({
                    "ID": cotizacion_id,
                    "Aseguradora": aseguradora,
                    "Cobertura": cobertura
                })
    
    # Crear DataFrame y escribir en hoja
    df_gastos = pd.DataFrame(gastos_data)
    for r_idx, row in enumerate(dataframe_to_rows(df_gastos, index=False, header=True), 1):
        ws_gastos.append(row)
    
    # Formatear encabezados
    for cell in ws_gastos[1]:
        cell.font = cell.font.copy(bold=True)
    
    # ======================================================
    # Eliminar hoja en blanco inicial y guardar
    # ======================================================
    if "Sheet" in wb.sheetnames:
        wb.remove(wb["Sheet"])
    
    # Ajustar ancho de columnas automáticamente
    def autoajustar_columnas(ws):
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column_letter].width = adjusted_width
    
    # Aplicar a todas las hojas
    for sheet in wb.sheetnames:
        autoajustar_columnas(wb[sheet])
    
    wb.save(output_file)
    return output_file

In [49]:
with open('../src/assets/2_elementos.json', 'r', encoding='utf-8') as f:
    datos = json.load(f)

# Generar Excel
archivo_excel = json_to_excel(datos, "reporte_seguros_2.xlsx")
print(f"Archivo Excel generado: {archivo_excel}")

/tmp/ipykernel_115905/2313050785.py:56: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)
/tmp/ipykernel_115905/2313050785.py:84: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)
/tmp/ipykernel_115905/2313050785.py:110: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)
/tmp/ipykernel_115905/2313050785.py:135: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)
/tmp/ipykernel_115905/2313050785.py:161: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)
/tmp/ipykernel_115905/2313050785.py:187: DeprecationWarning: Call to deprecated function cop

Archivo Excel generado: reporte_seguros_2.xlsx